In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('prac05').getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Read the CSV file into an RDD
df_b = spark.read.option("delimiter", ";").option("header", True).option('inferSchema', True).csv("/content/drive/MyDrive/Colab Notebooks/spark/bankmarketdata.csv")

df_b.printSchema()

df_b.show(5)

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+

In [ ]:
df_b.registerTempTable("bankings")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### 2.	Give marketing success rate. (No. of people subscribed / total no. of entries)

In [ ]:
q = '''
Select COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
From bankings
'''

spark.sql(q).show()

+--------------+-----------+---------+
|no_of_deposits|tot_no_cust|succ_rate|
+--------------+-----------+---------+
|          5289|      45211|     11.7|
+--------------+-----------+---------+



### 3.	Give marketing failure rate

In [ ]:
q = '''
Select COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate,
100 - round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as failure_rate
From bankings
'''

spark.sql(q).show()

+--------------+-----------+---------+------------+
|no_of_deposits|tot_no_cust|succ_rate|failure_rate|
+--------------+-----------+---------+------------+
|          5289|      45211|     11.7|        88.3|
+--------------+-----------+---------+------------+



### 4.	Maximum, Mean, and Minimum age of the average targeted customer

In [ ]:
q = '''
select max(age) as max_age, min(age) as min_age, 
round( avg(age), 2) as avg_age
from bankings
'''

spark.sql(q).show()

+-------+-------+-------+
|max_age|min_age|avg_age|
+-------+-------+-------+
|     95|     18|  40.94|
+-------+-------+-------+



### 5.	Check the quality of customers by checking the average balance, median balance of customers

In [ ]:
q = '''
Select round( avg(balance), 2) as avg_age,
round( percentile(balance, 0.5), 2) as median_age
From bankings
'''

spark.sql(q).show()

+-------+----------+
|avg_age|median_age|
+-------+----------+
|1362.27|     448.0|
+-------+----------+



### 6.	Check if age matters in marketing subscription for deposit

In [ ]:
q = '''
Select age, COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
From bankings
group by age
'''

spark.sql(q).show()

+---+--------------+-----------+---------+
|age|no_of_deposits|tot_no_cust|succ_rate|
+---+--------------+-----------+---------+
| 31|           206|       1996|    10.32|
| 85|             4|          5|     80.0|
| 65|            21|         59|    35.59|
| 53|            85|        891|     9.54|
| 78|            14|         30|    46.67|
| 34|           198|       1930|    10.26|
| 81|             6|         17|    35.29|
| 28|           162|       1038|    15.61|
| 76|            16|         32|     50.0|
| 27|           141|        909|    15.51|
| 26|           134|        805|    16.65|
| 44|            93|       1136|     8.19|
| 22|            40|        129|    31.01|
| 47|           113|       1088|    10.39|
| 52|            85|        911|     9.33|
| 86|             4|          9|    44.44|
| 40|           116|       1355|     8.56|
| 20|            15|         50|     30.0|
| 94|             0|          1|      0.0|
| 57|            78|        828|     9.42|
+---+------

In [ ]:
q = '''
select 
case
   when age < 20 then 'Under 20'
   when age between 21 and 30 then '21-30'
   when age between 31 and 40 then '31-40'
   when age between 41 and 50 then '41-50'
   when age between 51 and 60 then '51-60'
   when age > 60 then 'Over 60'
 END as age_range,
COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
 from bankings
 group by age_range
'''

spark.sql(q).show()

+---------+--------------+-----------+---------+
|age_range|no_of_deposits|tot_no_cust|succ_rate|
+---------+--------------+-----------+---------+
|     null|            15|         50|     30.0|
|  Over 60|           502|       1188|    42.26|
|    21-30|          1112|       6933|    16.04|
| Under 20|            18|         47|     38.3|
|    41-50|          1019|      11239|     9.07|
|    31-40|          1812|      17687|    10.24|
|    51-60|           811|       8067|    10.05|
+---------+--------------+-----------+---------+



### 7.	Check if marital status mattered for subscription to deposit.

In [ ]:
q = '''
Select marital, COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
From bankings
group by marital
'''

spark.sql(q).show()

+--------+--------------+-----------+---------+
| marital|no_of_deposits|tot_no_cust|succ_rate|
+--------+--------------+-----------+---------+
|divorced|           622|       5207|    11.95|
| married|          2755|      27214|    10.12|
|  single|          1912|      12790|    14.95|
+--------+--------------+-----------+---------+



In [ ]:
q = '''
select 
case
   when age < 20 then 'Under 20'
   when age between 21 and 30 then '21-30'
   when age between 31 and 40 then '31-40'
   when age between 41 and 50 then '41-50'
   when age between 51 and 60 then '51-60'
   when age > 60 then 'Over 60'
 END as age_range,marital,
COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
 from bankings
 group by age_range, marital
'''

spark.sql(q).show()

+---------+--------+--------------+-----------+---------+
|age_range| marital|no_of_deposits|tot_no_cust|succ_rate|
+---------+--------+--------------+-----------+---------+
|    21-30|divorced|            18|        170|    10.59|
|    41-50|divorced|           174|       1738|    10.01|
|  Over 60|divorced|            97|        199|    48.74|
|    31-40|divorced|           163|       1600|    10.19|
|    31-40| married|           927|      10114|     9.17|
|    31-40|  single|           722|       5973|    12.09|
|     null| married|             1|          3|    33.33|
|    51-60|divorced|           170|       1500|    11.33|
|    51-60| married|           588|       6072|     9.68|
|    51-60|  single|            53|        495|    10.71|
|  Over 60| married|           396|        955|    41.47|
| Under 20|  single|            18|         47|     38.3|
|     null|  single|            14|         47|    29.79|
|    21-30|  single|           913|       4705|     19.4|
|    21-30| ma

### 8.	Check if age and marital status together mattered for subscription to deposit scheme

In [ ]:
q = '''
Select age, marital, COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
From bankings
group by age, marital
'''

spark.sql(q).show()

+---+--------+--------------+-----------+---------+
|age| marital|no_of_deposits|tot_no_cust|succ_rate|
+---+--------+--------------+-----------+---------+
| 42|  single|            22|        218|    10.09|
| 55|  single|             0|         54|      0.0|
| 66| married|            22|         53|    41.51|
| 68|  single|             2|          4|     50.0|
| 28| married|            20|        325|     6.15|
| 59| married|            66|        583|    11.32|
| 61| married|            47|        123|    38.21|
| 21|  single|            21|         74|    28.38|
| 29|  single|           133|        683|    19.47|
| 70|divorced|             5|         13|    38.46|
| 56|  single|             6|         45|    13.33|
| 74| married|            11|         32|    34.38|
| 64|divorced|             4|          9|    44.44|
| 40|  single|            31|        257|    12.06|
| 45|  single|            14|        146|     9.59|
| 83|  single|             1|          2|     50.0|
| 69| marrie

In [ ]:
q = '''
Select age, marital, COUNT(IF(y = 'yes', 1, NULL)) as no_of_deposits,
count(*) as tot_no_cust,
round( (COUNT(IF(y = 'yes', 1, NULL)) / count(*)) * 100, 2) as succ_rate
From bankings
group by age, marital
'''

spark.sql(q).show()

In [ ]:
q = '''
select round( (sum(if(y='yes', 1, 0))/count(*) )*100,2) success_rate from bankings
'''

spark.sql(q).show()

+------------+
|success_rate|
+------------+
|        11.7|
+------------+



In [ ]:
df_agegrp = spark.sql(q)
df_agegrp.printSchema()

df_agegrp.write.csv("outputdf_agegrp")

root
 |-- age_range: string (nullable = true)
 |-- no_of_deposits: long (nullable = false)
 |-- tot_no_cust: long (nullable = false)
 |-- succ_rate: double (nullable = true)

